# Aggregating Data

You will undoubtedly want to aggregate your data in some fashion. You may be given the data in separate files or in various tables in a database. Some of the common aggregation tasks include:

- Querying
- Merging
- Binning
- Applying functions
- Summarizing
- Creating pivot tables and crosstabs

We will explore some of these tasks in this section.

## Querying Data

You will encounter two basic ways to query data when using `pandas`. One approach is to use a Boolean mask to get the results you want. The other approach is to use the `.query()` function. This function makes it easy to write complicated filters instead of using a Boolean mask. We will examine both approaches using weather data from Williamsburg found in the file `./data/wburg_weather.csv`.

In [ ]:
# Import pandas
import pandas as pd

In [ ]:
# Read in the cleaned_data from our last section
wburg_weather = pd.read_csv('./data/wburg_weather.csv')

# Look at its info()
wburg_weather.info()

We want to find all the hourly data where the wind direction was variable, the dry bulb temperature in Fahrenheit was between 75 and 80 degrees (inclusively), and the station pressure was less than 29.5. To use a Boolean mask, we use the bitwise **and** operator, which is the ampersand character, `&`. We also need to put each condition inside of parentheses. 

In [ ]:
# Using Boolean mask
wburg_weather[(wburg_weather.HOURLYWindDirection == 'Variable Direction')
             & (wburg_weather.HOURLYDRYBULBTEMPF >= 75) 
             & (wburg_weather.HOURLYDRYBULBTEMPF <= 80)
             & (wburg_weather.HOURLYStationPressure < 29.5)]

When we use the `.query()` method, we can use both logical operators (`and`, `or`, `not`) and bitwise operators (`&`, `|`, `~`). In our case, we can use the logical operator `and`, which makes reading our statement easier. You should be able to verify that the results of both approaches are the same.

In [ ]:
# Use the .query() method instead
wburg_weather.query('HOURLYWindDirection == "Variable Direction" '
                   'and HOURLYDRYBULBTEMPF >= 75 and HOURLYDRYBULBTEMPF <= 80 '
                   'and HOURLYStationPressure < 29.5')

<hr style="border:1px solid gray">

<font color='red' size = '5'> Student Exercise </font>

Continuing with our weather data, complete the following tasks:

1. Using a Boolean mask, find all the rows that have hourly sky conditions of `CLR:00`, a dew point temperature greater than 75, and the relative humidity of either 94 or 100. Save your results in the variable `my_mask`.
2. How many rows does `my_mask` have?
3. Using the `.query()` method, perform the same task as above. Save the results in the variable `my_query`.
4. How many rows does `my_query` have?
5. Verify that the two `DataFrame`s contain the same data.

In [ ]:
# 1. Using a Boolean mask, find all the rows that have hourly sky conditions of `CLR:00`,
# a dew point temperature greater than 75, and the relative humidity of either 94 or 100.
# Save your results in the variable `my_mask`.


In [ ]:
# 2. How many rows does `my_mask` have?


In [ ]:
# 3. Using the `.query()` method, perform the same task as above. 
# Save the results in the variable `my_query`.


In [ ]:
# 4. How many rows does `my_query` have?


In [ ]:
# 5. Verify that the two `DataFrame`s contain the same data.


<hr style="border:1px solid gray">

## Binning

There are times when you want to take a variable that is continuous (or discrete with a lot of valid values) and create a new variable that creates "buckets" (bins or categories). For example, you might be more interested in analyzing the daily high temperature as one of several categories instead of a numerical quantity. Perhaps you think any temperature above 90&deg; F is "hot". You could create five different "buckets" to classify temperature as "cold", "cool", "just right", "warm", or "hot". How you decided to discretize a continuous variable is highly dependent on how you plan to use the new variable in your analysis. Here we will explore a couple of ways to bin our data.

We will be using the daily stock trading data for the company Apple for the year 2022. The data is in the file `./data/aapl_2022.csv`. Let's read it in trying to take care of the commas in the numerical columns and being sure to create a real date for the `Date` column.

In [ ]:
# Read in the Apple trading data
apple = pd.read_csv('./data/aapl_2022.csv', thousands=',', parse_dates=['Date'])
apple.info()

In [ ]:
# Change all column names to lower case
apple.columns = apple.columns.str.lower()
apple.info()

In [ ]:
# Change the index to the date
apple.set_index('date', inplace=True)
apple.info()

We are interested in investigating the volume of trading for Apple's stock in the year 2022. In particular, we want to calculate the absolute value of the Z-score for volume. Recall from statistics that the Z-score tells you how far away a particular observation is from the mean of all the observations for that variable. It tells you how many standard deviations away a data point is from the average. If your data follows a Normal distribution, then you would expect approximately 99.7% of the observations to fall between -3 and +3 standard deviations from the mean. Let's see if we have any days of trading volume that are outside of three standard deviations, in absolute terms, for Apple.

In [ ]:
# Find the absolute value of Z-score for volume
apple = apple.assign(abs_z_score_vol 
                     = lambda x: x.volume.sub(x.volume.mean()).div(x.volume.std()).abs())

In [ ]:
# Which days had volumes outside of 3 standard deviations?
apple.query('abs_z_score_vol > 3')

In [ ]:
# Examine trading days around the May 12th date
apple.loc['2022-05-10':'2022-05-16']

You may also be interested in the percentage change of a variable. You can use the `.pct_change()` function on either the `DataFrame` or a `Series`. This function will calculate the percentage change from the immediately preceding row by default. In our case with time series data, this is the action we want. In addition to finding the percentage change, we are also going to create a column that ranks the percentage change. The ranking column allows us to sort the data more easily.

In [ ]:
# See the percent change for volume and rank it
# This returns a new DataFrame of the 10 largest
# volume changes in percentage change from the previous trading day
apple.assign(vol_pct_change = apple.volume.pct_change(),
            pct_change_rank=lambda x: x.vol_pct_change.abs().rank(ascending=False)
            ).nsmallest(10, 'pct_change_rank')

Now we want to try to create bins or buckets for the daily trading volume. If you want the ranges defining the bins to be equidistant, then you can use the `pd.cut()` function. Let's try breaking the column `volume` into three equidistant bins. We can use the argument `bins=3` and `pandas` will find the break points for the bins for us. You can optionally label each bin, which we will do as "low", "medium", and "high". If you want both the binned data as well as the ranges, you use the argument `retbins=True`. The first element of the returned tuple will be the binned data and the second element will contain the defining break points for the bins. 

In [ ]:
# Unpack the returned tuple
# First element is the data that has been binned
# Second element is the break points of the bins
vol_binned, bin_ranges = pd.cut(apple.volume, bins=3,
                                labels=['low','medium','high'],
                                retbins=True)

In [ ]:
# See the break points for the bins
bin_ranges

In [ ]:
# See the range for each of 3 bins
for i in range(len(bin_ranges)-1):
    print(f'size of bin is: {bin_ranges[i+1] - bin_ranges[i]}')

In [ ]:
# See how many rows are in each bucket
vol_binned.value_counts()

In [ ]:
# Look at the 13 high volume days
apple[vol_binned=='high'].sort_values('volume', ascending=False)

Another way to create buckets is to try to have the same number of observations in each bucket. Doing so necessitates that the breakpoints for the bins **not** be equidistant. We can accomplish breaking a continuous variable into a discretized variable with each category containing (roughly) the same number of observations with the `pd.qcut()` function. The `q` stands for **quantile**. Using the argument `q=4` will give you back the quartiles; using `q=10` will return deciles. Let's break the `volume` variable into quartiles.

In [ ]:
# Equal number of observations in each bin
# Unpack the returned tuple
# First element is the data that has been binned
# Second element is the break points of the bins
vol_qbinned, qbin_ranges = pd.qcut(apple.volume, q=4,
                                   labels=['q1','q2','q3','q4'],
                                   retbins=True)

In [ ]:
# How many in each bin? Should be approximately same
vol_qbinned.value_counts()

In [ ]:
# See the break points for qcut
qbin_ranges

In [ ]:
# See the range for each of 4 bins (quartiles)
for i in range(len(qbin_ranges)-1):
    print(f'size of bin is: {qbin_ranges[i+1] - qbin_ranges[i]}')


In [ ]:
# Call .describe on the volume
# The 5 numbers for the qcut should be readily apparent
apple.volume.describe()

<hr style="border:1px solid gray">

## Summarizing `DataFrame`s

There are times when we want to summarize various columns of a `DataFrame`. Sometimes you want the same summary statistics for all the numerical columns. Other times, you might want different summary statistics for different columns. If you want the same summary statistic, e.g., the average of each column, you can use the `.apply()` function discussed above. Remember that each column data type must be in a form that allows the desired function to complete its action. If you want to find different summary statistics for different columns, the `.agg()` function can help. 

Because we added text columns to our `apple` dataset, calling `apple.apply(np.mean)` or something similar will fail. So, let's subset our `apple` dataset and then try using `.apply()` on the entire `DataFrame`.

In [ ]:
# Import numpy with alias np
import numpy as np

In [ ]:
# Change the output format for floats to be 2 decimals
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [ ]:
# Get rid of nonnumerical columns
only_num_apple = apple.select_dtypes(include=np.number)
only_num_apple.info()

In [ ]:
# Find the average for all columns with .apply()
only_num_apple.apply(np.mean)

In [ ]:
# Now use .agg to find different summary statistics
# for different columns. Because picking out columns
# you can use the apple dataset
apple.agg({'open':np.mean, 'high':np.max, 'low':np.min,
          'close':np.mean, 'volume':np.sum})

In [ ]:
# If you want to use more than one statistic
# for a column, pass them as a list
apple.agg({'open':'mean',
          'high':['min','max'],
          'low':['min','max'],
          'close':'mean'})

<hr style="border:1px solid gray">

## Aggregating by Group

There are many instances where you want to calculate aggregations per group. To do so, we can use the `.groupby()` function, passing in the column(s) we want to use to determine distinct groups. Let's add the trading volume groups to our `apple` dataset. We will use the `pd.cut()` function as we did earlier.

In [ ]:
# Add the binned groups to the data
apple = apple.assign(vol_group = pd.cut(apple.volume, bins=3,
                   labels=['low','medium','high']))

apple.info()         

In [ ]:
# Let's find the average for each  vol_group
apple.groupby('vol_group').mean()

In [ ]:
# We can also pick out one of the columns from the resulting
# groupby and do specific aggregations on that column
apple.groupby('vol_group')['close'].agg(['min','max','mean'])

In [ ]:
# If you want to pull out different summary statistics
# for different columns, you can use .agg
app_agg = apple.groupby('vol_group').agg({
    'open':'mean', 'high':['min','max'],
    'low':['min','max'], 'close':'mean'})

app_agg

In [ ]:
# See the columns of app_agg
app_agg.columns

In [ ]:
# If you want to "flatten" the columns, one approach
# is to use list comprehension as shown here
app_agg.columns = ['_'.join(col_agg) for col_agg in app_agg.columns]
app_agg

You can also use a `pd.Grouper`, which allows you to specify a groupby instruction for an object. This is especially useful when you have an index that is a date. Our `apple` dataset has a date for an index. Suppose we wanted to find the average for the open, high, low, and close stock price for each month. This is easily accomplished with a `Grouper` by specifying the frequency. 

In [ ]:
# Find the aveage open, high, low, and close price for each month
apple.groupby(pd.Grouper(freq='M'))[['open','high','low','close']].mean()

<hr style="border:1px solid gray">

## Pivot Tables and Crosstabs

You are probably familiar with the powerful concept of pivot tables from Excel. We can quickly generate pivot tables or cross tabulations in a commonly used format with `pandas`. You need to specify what to group on and, optionally, which subset of columns we want to aggregate and/or how to aggregate. The default aggregation is **average**.

In [ ]:
# Call pivot_table with a grouping vol_group
apple.pivot_table(columns='vol_group')

If you want the transpose of this pivot table, you can specify `index='vol_group'` instead. Let's see if it works.

In [ ]:
# Put the vol_group categories on the rows
apple.pivot_table(index='vol_group')

The function `pd.crosstab()` can be used to create a frequency table. For example, suppose we want to know how many low-, medium-, and high-volume trading days occurred in each month of our dataset. You use `index` to specify the rows and `columns` to specify the columns. By default, the values in the cells will be the count.

In [ ]:
# Create a frequency table
# Rows will be the vol_group
# Columns will be the month number
pd.crosstab(index=apple.vol_group, columns=apple.index.month,
           colnames=['month'])

In [ ]:
# If you want totals for the rows and columns
# you can use the argument margins=True
pd.crosstab(index=apple.vol_group, columns=apple.index.month,
           colnames=['month'], margins=True)

In [ ]:
# If you want percentages by the month, you can use
# the argument normalize='columns'
pd.crosstab(index=apple.vol_group, columns=apple.index.month,
           colnames=['month'], normalize='columns')

In [ ]:
# If you want the overall percentage for each group over 
# the year, you can use normalize='columns' and margins=True
pd.crosstab(index=apple.vol_group, columns=apple.index.month,
           colnames=['month'], normalize='columns', margins=True)

<hr style="border:1px solid gray">

### Additional Resources

The following links point you to additional resources that you might find helpful in learning this material.

1. [The official API reference for `pandas.DataFrame.query`][1].
2. [The official API reference for `pandas.Series.pct_change`][2].
3. [The official API reference for `pandas.cut`][3].
4. [The official API reference for `pandas.qcut`][4].
5. [The official API reference for `pandas.Grouper`][5].

-----

[1]: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.query.html
[2]: https://pandas.pydata.org/docs/reference/api/pandas.Series.pct_change.html
[3]: https://pandas.pydata.org/docs/reference/api/pandas.cut.html
[4]: https://pandas.pydata.org/docs/reference/api/pandas.qcut.html
[5]: https://pandas.pydata.org/docs/reference/api/pandas.Grouper.html

-----

**&copy; 2022 - Present: Matthew D. Dean, Ph.D.   
Clinical Associate Professor of Business Analytics at William \& Mary.**